# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

In [2]:
! pip install pandas
! pip install regex
! pip install matplotlib
! pip install nltk



[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [3]:
! pip install sklearn

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 38.3 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 35.4 MB/s eta 0:00:00m eta 0:00:010:01:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1304 sha256=c15193925e219937d54d441f78d7c2be3ca42f42095ac4070f19e9db6e41c303
  Stored in directory: /home/gitpod/.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [4]:

import pandas as pd 
import regex as reg
import re
import matplotlib.pyplot as plt
import unicodedata
import nltk
import string
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from sklearn import model_selection, svm
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords


In [5]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv')

In [6]:
df_raw.info

<bound method DataFrame.info of                                                     url  is_spam
0     https://briefingday.us8.list-manage.com/unsubs...     True
1                                https://www.hvper.com/     True
2                    https://briefingday.com/m/v4n3i4f3     True
3      https://briefingday.com/n/20200618/m#commentform    False
4                           https://briefingday.com/fan     True
...                                                 ...      ...
2994  https://www.smartcitiesworld.net/news/news/dee...    False
2995                      https://www.youtube.com/watch     True
2996  https://techcrunch.com/2019/07/04/an-optimisti...    False
2997  https://www.technologyreview.com/2019/12/20/13...    False
2998       https://www.bbc.com/news/technology-51018758    False

[2999 rows x 2 columns]>

In [7]:
#Vamos a pasar a 0 y 1 la variable objetivo

df_raw['is_spam'] = df_raw['is_spam'].apply(lambda x: 1 if x == True else 0)

In [8]:
df_raw.info

<bound method DataFrame.info of                                                     url  is_spam
0     https://briefingday.us8.list-manage.com/unsubs...        1
1                                https://www.hvper.com/        1
2                    https://briefingday.com/m/v4n3i4f3        1
3      https://briefingday.com/n/20200618/m#commentform        0
4                           https://briefingday.com/fan        1
...                                                 ...      ...
2994  https://www.smartcitiesworld.net/news/news/dee...        0
2995                      https://www.youtube.com/watch        1
2996  https://techcrunch.com/2019/07/04/an-optimisti...        0
2997  https://www.technologyreview.com/2019/12/20/13...        0
2998       https://www.bbc.com/news/technology-51018758        0

[2999 rows x 2 columns]>

In [9]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [10]:
df_raw['url'].sample(20)

1555    https://www.apollo-magazine.com/future-museum-...
881     https://www.cnbc.com/2020/06/24/imf-global-eco...
2226    https://www.marketwatch.com/story/another-138-...
2079    http://www.bbc.com/travel/story/20180621-the-u...
828     https://www.vox.com/science-and-health/2097828...
2748       https://www.planbonestep.com/how-plan-b-works/
785           https://qz.com/guide/chinas-africa-project/
1782    https://arstechnica.com/gaming/2020/06/doomscr...
1708    https://aeon.co/essays/where-do-children-s-ear...
1388    https://www.designbetter.co/podcast/nancy-douyon/
2752    https://www.nbcnews.com/feature/nbc-out/n-y-c-...
949     https://apnews.com/e32097aa4318b3eabf00f785d30...
1913                   https://benjaminreinhardt.com/wddw
1584    https://www.cnbc.com/2020/06/26/nike-ceo-john-...
24                      http://josefschulz.com/uebergang/
803      https://bigspaceship.us9.list-manage.com/profile
269     https://www.theguardian.com/us-news/2020/jun/2...
2769    https:

In [11]:
df_raw['is_spam'].value_counts()

0    2303
1     696
Name: is_spam, dtype: int64

In [12]:
# Vamos a ver como esta nuestra variable objetivo en este caso es is_spam
df_raw['is_spam'].value_counts()
print("Hay spam: ",len(df_raw.loc[df_raw.is_spam==1]))
print("Que no son spam : ",len(df_raw.loc[df_raw.is_spam==0]))
print(df_raw.shape)


Hay spam:  696
Que no son spam :  2303
(2999, 2)


In [13]:
# Eliminar los duplicados

df_raw = df_raw.drop_duplicates()
df_raw = df_raw.reset_index(inplace = False)[['url','is_spam']]
df_raw.shape

(2369, 2)

Empezamos el proceso de Limipieza de los datos

In [14]:
# colocar todos los textos en minusculas
df_raw['url'] = df_raw['url'].str.lower()

In [15]:
df_aux = df_raw.copy()

In [16]:

# Proceso de limpieza de datos

limpieza = []

for p in range(len(df_aux.url)):
    desc = df_aux['url'][p]
    
    #sacar la puntuacion
    desc = re.sub('[^a-zA-Z]', ' ', desc)
    
    #borrar etiquetas especiales
    desc=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",desc)
    
    #borrar digitos y caracteres especiales
    desc=re.sub("(\\d|\\W)+"," ",desc)
    
    limpieza.append(desc)

#asiganmos los datos limipios en limpieza
df_aux['url'] = limpieza
        
df_aux.head()

,url,is_spam
0,https briefingday us list manage com unsubscribe,1
1,https www hvper com,1
2,https briefingday com m v n i f,1
3,https briefingday com n m commentform,0
4,https briefingday com fan,1


In [17]:
df_aux['url'].str.split(expand=True).stack().value_counts()[:60]

https          2336
com            2065
www            1512
the             354
html            296
news            274
a               252
us              248
to              218
of              173
coronavirus     172
e               150
org             146
c               136
article         131
b               124
in              115
and             113
morningbrew     105
story           105
nytimes         101
on              101
daily            99
d                98
stories          94
utm              90
for              90
youtube          89
v                89
trump            88
numlock          87
watch            86
f                78
is               77
new              76
p                69
world            68
substack         68
reuters          65
covid            63
s                62
briefingday      61
index            61
vox              59
en               59
articles         58
cnn              58
iduskbn          58
politics         56
co               56


In [18]:
# Creamos una lista de stopwords
stop_words = ['http','www','com','you','your','for','not','have','is','in','im','from','to','https','e','c','v','b','f','p']

In [19]:

for i in df_aux['url'].str.split(expand=True).stack().value_counts().index:
    if len(i)<3 :
        stop_words.append(i)


In [20]:
stop_words=list(set(stop_words))
len(stop_words)

268

In [21]:

def remove_stopwords(message):
  if message is not None:
    words = message.strip().split()
    words_filtered = []
    for word in words:
      if word not in stop_words:
        words_filtered.append(word) 
    result = " ".join(words_filtered)         
  else:
    result = None

  return result 


In [22]:
df_aux['url']=df_aux['url'].apply(remove_stopwords)

In [23]:
df_aux.sample(10)

,url,is_spam
1778,techcrunch india bans tiktok dozens other chin...,0
2044,mailchi bigspaceship big spaceships internet b...,0
1053,simplisafe value,1
312,imdb title,0
2209,thesun news uknews euromillions adrian bayford...,0
992,staykeen landing,0
1594,melmagazine story waterloo brought abba the wo...,0
438,consequenceofsound net grateful dead deodorant,0
866,change name turok,0
2246,numlock substack numlock sunday jim ottavianti...,1


In [24]:
df_aux['url'].str.split(expand=True).stack().value_counts()[:60]

the            354
html           296
news           274
coronavirus    172
org            146
article        131
and            113
morningbrew    105
story          105
nytimes        101
daily           99
stories         94
utm             90
youtube         89
trump           88
numlock         87
watch           86
new             76
substack        68
world           68
reuters         65
covid           63
briefingday     61
index           61
vox             59
cnn             58
articles        58
iduskbn         58
politics        56
cnbc            54
sunday          51
business        49
court           48
apnews          47
email           46
facebook        46
health          45
bbc             41
supreme         41
blog            40
are             40
black           39
medium          39
npr             38
police          38
digg            37
campaign        37
with            37
why             36
theverge        35
apple           35
cases           35
theguardian 

In [25]:
# Ahora que hemos limpiado nuestros datos
# procedemos a hacer una copia con el dataset a trabajar en el final
df = df_aux.copy()

In [26]:
X = df['url']
y = df['is_spam']
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=121)

In [27]:
# Vectorizador
vec = CountVectorizer(stop_words='english')
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.transform(X_test).toarray()

In [28]:
print(X_train)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [29]:
X_train.shape

(1776, 4658)

In [30]:
X_test.shape

(593, 4658)

In [31]:
print(vec.get_feature_names_out())

['aab' 'aaron' 'abacus' ... 'zulalimtm' 'zwift' 'zwn']


In [32]:
nb = MultinomialNB()

In [33]:
nb.fit(X_train, y_train)

MultinomialNB()

In [34]:
predictions = nb.predict(X_train)
print(classification_report(y_train, predictions))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1593
           1       0.84      0.69      0.76       183

    accuracy                           0.95      1776
   macro avg       0.90      0.84      0.87      1776
weighted avg       0.95      0.95      0.95      1776



In [35]:
predictions = nb.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.93      0.98      0.96       532
           1       0.70      0.38      0.49        61

    accuracy                           0.92       593
   macro avg       0.81      0.68      0.72       593
weighted avg       0.91      0.92      0.91       593



In [36]:
#Creando la Matriz

message_vectorizer = CountVectorizer().fit_transform(df['url'])

# Haciendo el split de los datos

X_train, X_test, y_train, y_test = train_test_split(message_vectorizer, df['is_spam'], test_size = 0.2, random_state = 121, shuffle = True)

In [37]:
X_train.shape

(1895, 5775)

In [38]:
cl = svm.SVC(C=1.0, kernel='linear', degree=4, gamma='auto')

In [39]:
cl.fit(X_train, y_train)

pred = cl.predict(X_train)
print(classification_report(y_train, pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1701
           1       0.99      0.92      0.96       194

    accuracy                           0.99      1895
   macro avg       0.99      0.96      0.98      1895
weighted avg       0.99      0.99      0.99      1895



In [40]:
pred = cl.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.95      0.96      0.96       424
           1       0.66      0.58      0.62        50

    accuracy                           0.92       474
   macro avg       0.81      0.77      0.79       474
weighted avg       0.92      0.92      0.92       474



Este último es el mejor resultado obtenido, por lo que considero guardar

In [41]:
pickle.dump(cl, open('../models/texto_NLP.pkl', 'wb'))

In [42]:
# Si queremos cargar el archivo guardado en la carpeta models

load_model = pickle.load(open('../models/texto_NLP.pkl', 'rb'))

In [43]:
load_model

SVC(degree=4, gamma='auto', kernel='linear')

Ahora probe solo cambiando a las columnas, y el resultado al final no es el mejor

In [44]:
df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv')

In [45]:
df['len_url'] = df['url'].apply(lambda x : len(x))
df['contains_subscribe'] = df['url'].apply(lambda x : 1 if "subscribe" in x else 0)
df['contains_hash'] = df['url'].apply(lambda x : 1 if "#" in x else 0)
df['num_digits'] = df['url'].apply(lambda x : len("".join(_ for _ in x if _.isdigit())) )
df['non_https'] = df['url'].apply(lambda x : 1 if "https" in x else 0)
df['num_words'] = df['url'].apply(lambda x : len(x.split("/")))

target = 'is_spam'
features = [f for f in df.columns if f not in ["url", target]]
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=0)

In [46]:
X_train.head()

,len_url,contains_subscribe,contains_hash,num_digits,non_https,num_words
1521,74,0,0,0,1,6
1473,34,0,0,0,1,5
731,114,0,0,12,1,8
271,51,0,0,20,1,4
1509,43,0,0,0,1,5


In [47]:
df['contains_hash'].value_counts()

0    2943
1      56
Name: contains_hash, dtype: int64

In [48]:
cl = svm.SVC(C=1.0, kernel='linear', degree=4, gamma='auto')

In [49]:
cl.fit(X_train, y_train)
pred = cl.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

       False       0.91      0.90      0.91       464
        True       0.67      0.70      0.69       136

    accuracy                           0.85       600
   macro avg       0.79      0.80      0.80       600
weighted avg       0.86      0.85      0.86       600



Esta prueba no es la mejor, pero el tema que tenemos mas columnas con las palabras separadas que cantidad de observaciones
lo que se hace dificil predecir algo en este caso